## Part 1

1) For both Smith-Waterman and Needleman-Wunsch algorithms:
    a) What are the parameters and variables required for algorithm initialization, execution, and termination?
    b) What quantities are returned?
    c) What is the runtime complexity?
    
    Smith-Waterman:
        a) Parameters:
            Initialization:
                a) seq1: First protein sequence
                b) seq2: Second protein sequence
                c) smat: scoring matrix
            Execution:
                a) scores: matrix of size M x N that records scores of optimal alignment
                b) traceback: matrix of size M x N that records previous cell from which each cell was computed from
            Termination:
                a) result: optimal alignment string
        b) Returns: Optimal sub-alignment with positive scores
        c) Complexity: O(MxN)

    Needleman-Wunsch:
        a) Parameters:
            Initialization:
                a) seq1: First protein sequence
                b) seq2: Second protein sequence
                c) smat: scoring matrix
            Execution:
                a) scores: matrix of size M x N that records scores of optimal alignment
                b) traceback: matrix of size M x N that records previous cell from which each cell was computed from
            Termination:
                a) result: optimal alignment string
        b) Returns: Full alignment string of sequence 
        c) Complexity: O(MxN)

2) What functionalities in initialization, execution and termination are shared between these algorithms? Which are not shared?

    Initialization: Both methods initialize and M x N matrix where M is the length of seq1 and N is the length of seq2. However, in initializing the SW matrix, you set the first row and first column to 0. 
    
    Execution: Both methods share methods for calculating the score at a particular cell in the matrix. This entails taking the max of the three following scores:
        a) Gap from left to right
        b) Gap from top to bottom
        c) Match
    However, SW adds an extra option where you can reset to 0 if the score goes negative.
    
    Termination: Both methods have to use the traceback matrix to return an alignment. NW must start at the lower rightmost cell and trace back all the way to the start and return that string. SW needs to start at the max element in the matrix and trace back until a 0 is reached, then return the resulting string.

3) How does affine-gap based alignment differ from linear-gap alignment in terms of implementation?
    
    Affine gap alignment requires you to keep track of the gap status of the previous cell when computing the score of the current cell (gap start, match/mismatch, gap extension). To return the optimal alignment you need two extra score matrices, one for seq1 and one for seq2, which keep track of the max score of either opening or extending a gap at the current cell. For the final traceback you would need to interface between the original score matrix and these two extra matrices to get the alignment.

In [ ]:
f = open("./sequences/prot-0088.fa",'r').read()
print(f)

In [ ]:
sw = SmithWaterman("/Users/mtsui1/Documents/Classes/Algs/Project1/scoring_matrices/BLOSUM50.mat")
scores, alignment = sw.align("./sequences/test1.fa", "./sequences/test2.fa")
print(scores)
print(alignment)

## Part 2

In [3]:
# read Pospairs and negpairs into a list

f = open("./scoring_matrices/Pospairs.txt",'r')
lines = f.readlines() # skip header
pos_pairs = []
for line in lines:
    line = line.strip()
    line = [i for i in line.split(" ") if i != ""]
    pos_pairs.append(line)
    
f = open("./scoring_matrices/Negpairs.txt",'r')
lines = f.readlines() # skip header
neg_pairs = []
for line in lines:
    line = line.strip()
    line = [i for i in line.split(" ") if i != ""]
    neg_pairs.append(line)

In [36]:
from align.algs import *

In [ ]:
sw = SmithWaterman("/Users/mtsui1/Documents/Classes/Algs/Project1/scoring_matrices/BLOSUM50.mat")

In [ ]:
sw.set_gap_open(-11)
sw.set_gap_extend(-3)

In [4]:
len(pos_pairs)

50

In [5]:
len(neg_pairs)

50

In [ ]:
pos_pairs[0][0]

In [ ]:
pos_scores = {}
for pair in pos_pairs:
    print(pair)
    score = sw.align(pair[0], pair[1], return_alignment=False)

    pos_scores[(pair[0], pair[1])] = score

In [ ]:
pos_scores

In [ ]:
sw = algs.NeedlemanWunsch("/Users/mtsui1/Documents/Classes/Algs/Project1/scoring_matrices/BLOSUM50.mat")
#sw.set_gap_open(-11)
#sw.set_gap_extend(-3)
score, alignment = sw.align("sequences/test1.fa", "sequences/test2.fa")
print(score)
print(alignment)

In [ ]:
neg_scores = {}
for pair in neg_pairs:
    print(pair)
    score = sw.align(pair[0], pair[1], return_alignment=False)

    neg_scores[(pair[0], pair[1])] = score

In [ ]:
neg_scores

In [ ]:
p_scores = list(pos_scores.values())

n_scores = list(neg_scores.values())

all_scores = p_scores + n_scores
print(all_scores)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(x=all_scores, bins='auto', rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Distribution of aligment scores')

The distribution is centered around scores from ~25-50, meaning that most alignments are falling into this score range. There are some higher scoring outliers as well.

In [ ]:
import pandas as pd

avg_score = np.mean(all_scores)
print("Average score (threshold): " + str(avg_score))

# predict that the scores higher than threshold are True
predicted = all_scores > avg_score

# First 50 scores are true positive, last 50 are true negative
actual = list(np.repeat(True, 50))  + list(np.repeat(False, 50))

df = pd.DataFrame({'predicted': predicted, 'actual': actual})


In [ ]:
confusion_matrix = pd.crosstab(df['actual'], df['predicted'], rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

In [ ]:
#The sensitivity is TP/TP+FN
sensitivity = 22/(22+28)
print("sensitivity: " + str(sensitivity))

# specificity = TN/TN+FP
specificity = 44/(44+6)
print("specificity: " + str(specificity))

Using the average score to threshold seems to give a low sensitivity but high specificity. I notice that there are lots of false negatives. 

In [19]:
from sklearn.metrics import roc_auc_score, roc_curve

fpr, tpr, thresholds = roc_curve(df['actual'], df['predicted'])
ns_probs = [False for _ in range(len(df['predicted']))]
ns_fpr, ns_tpr, _ = roc_curve(df['actual'], ns_probs)

fig = plt.figure()
ax = fig.add_subplot(111)

plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='Local alignment')

plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
ax.set_aspect('equal', adjustable='box')


NameError: name 'df' is not defined

In [ ]:
from sklearn.metrics import roc_auc_score

# calculate AUC
auc = roc_auc_score(df['actual'], df['predicted'])
print("AUC: " + str(auc))

From the AUC, using the average score as a threshold, you can see that algorithm performs at least better than the random line (AUC 0.5). However, you cannot determine the performance using just one threshold. You need to try many different thresholds to find the one that maximizes the AUC.

In [2]:
sw = SmithWaterman("/Users/mtsui1/Documents/Classes/Algs/Project1/scoring_matrices/BLOSUM62.mat")

In [6]:
pos_scores_gaps = {}

for gap_open in range(1,21):
    sw.set_gap_open(-gap_open)
    for gap_extend in range(1,6):
        scores = []
        sw.set_gap_extend(-gap_extend)
        for pair in pos_pairs:
            score = sw.align(pair[0], pair[1], return_alignment=False)
            scores.append(score)
        pos_scores_gaps[(gap_open, gap_extend)] = scores
        print(gap_open, gap_extend)


1 1
1 2
1 3
1 4
1 5
2 1
2 2
2 3
2 4
2 5
3 1
3 2
3 3
3 4
3 5
4 1
4 2
4 3
4 4
4 5
5 1
5 2
5 3
5 4
5 5
6 1
6 2
6 3
6 4
6 5
7 1
7 2
7 3
7 4
7 5
8 1
8 2
8 3
8 4
8 5
9 1
9 2
9 3
9 4
9 5
10 1
10 2
10 3
10 4
10 5
11 1
11 2
11 3
11 4
11 5
12 1
12 2
12 3
12 4
12 5
13 1
13 2
13 3
13 4
13 5
14 1
14 2
14 3
14 4
14 5
15 1
15 2
15 3
15 4
15 5
16 1
16 2
16 3
16 4
16 5
17 1
17 2
17 3
17 4
17 5
18 1
18 2
18 3
18 4
18 5
19 1
19 2
19 3
19 4
19 5
20 1
20 2
20 3
20 4
20 5


In [11]:
new = {}
for k,v in pos_scores_gaps.items():
    new[str(k)] = v
    

In [12]:
with open('./pos_scores_gaps.json', 'w') as f:
    json.dump(new, f)

In [26]:
with open('./pos_scores_gaps.json') as f:
    pos_scores_gaps = json.load(f)

In [16]:
neg_scores_gaps = {}

for gap_open in range(1,21):
    sw.set_gap_open(-gap_open)
    for gap_extend in range(1,6):
        scores = []
        sw.set_gap_extend(-gap_extend)
        for pair in neg_pairs:
            score = sw.align(pair[0], pair[1], return_alignment=False)
            scores.append(score)
        neg_scores_gaps[str((gap_open, gap_extend))] = scores
        print(gap_open, gap_extend)


1 1
1 2
1 3
1 4
1 5
2 1
2 2
2 3
2 4
2 5
3 1
3 2
3 3
3 4
3 5
4 1
4 2
4 3
4 4
4 5
5 1
5 2
5 3
5 4
5 5
6 1
6 2
6 3
6 4
6 5
7 1
7 2
7 3
7 4
7 5
8 1
8 2
8 3
8 4
8 5
9 1
9 2
9 3
9 4
9 5
10 1
10 2
10 3
10 4
10 5
11 1
11 2
11 3
11 4
11 5
12 1
12 2
12 3
12 4
12 5
13 1
13 2
13 3
13 4
13 5
14 1
14 2
14 3
14 4
14 5
15 1
15 2
15 3
15 4
15 5
16 1
16 2
16 3
16 4
16 5
17 1
17 2
17 3
17 4
17 5
18 1
18 2
18 3
18 4
18 5
19 1
19 2
19 3
19 4
19 5
20 1
20 2
20 3
20 4
20 5


In [17]:
len(neg_scores_gaps)

100

In [18]:
with open('./neg_scores_gaps.json', 'w') as f:
    json.dump(neg_scores_gaps, f)

In [29]:
import pandas as pd

aucs = {}

# First 50 scores are true positive, last 50 are true negative
actual = list(np.repeat(True, 50))  + list(np.repeat(False, 50))

for k,v in pos_scores_gaps.items():
    all_scores = pos_scores_gaps[k] + neg_scores_gaps[k]
    avg_score = np.mean(all_scores)
    pred_scores = all_scores > avg_score
    auc = roc_auc_score(actual, pred_scores)
    aucs[k] = auc



In [34]:
maximum = max(aucs, key=aucs.get)  # Just use 'min' instead of 'max' for minimum.
print(maximum, aucs[maximum])
# D 87

(2, 3) 0.7400000000000001


In [ ]:
sw.set

In [ ]:
pos_scores = {}
for pair in pos_pairs:
    print(pair)
    score = sw.align(pair[0], pair[1], return_alignment=False)

    pos_scores[(pair[0], pair[1])] = score